In [1]:
import os

if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')
    
os.getcwd()

'/home/zack/personal/prediction_market_tools'

In [2]:
import pandas as pd

In [3]:
p = pd.read_csv('data/raw/predsnprices.csv')

In [4]:
from src import expected_value

In [5]:
p, latest_p = expected_value.prep_prices_csv(p)

In [6]:
p

,symbol,date,yesPrice,noPrice,myBet,purchase,qty
0,lcase-001,2021-09-02,0.58,0.44,0.72,NaN,NaN
1,lcase-001,2021-09-09,0.19,0.82,0.35,NaN,NaN
2,voch-001,2021-09-09,0.35,0.66,0.25,NaN,NaN
3,nyschl-001,2021-09-09,0.84,0.18,0.90,NaN,NaN
4,wkcase-004,2021-09-13,0.57,0.45,0.63,NaN,NaN
5,lcase-001,2021-09-14,0.11,0.91,0.15,NaN,NaN
6,voch-001,2021-09-14,0.36,0.66,0.25,n,98.0
7,nyschl-001,2021-09-14,0.84,0.17,0.90,y,125.0
8,wkcase-004,2021-09-14,0.74,0.29,NaN,NaN,NaN
9,case-038,2021-09-14,0.07,0.95,0.05,NaN,NaN


In [7]:
positions = p[~p.purchase.isna()]

In [8]:
positions

,symbol,date,yesPrice,noPrice,myBet,purchase,qty
6,voch-001,2021-09-14,0.36,0.66,0.25,n,98.0
7,nyschl-001,2021-09-14,0.84,0.17,0.90,y,125.0


In [9]:
positions['price'] = positions.yesPrice.where(positions.purchase == 'y', positions.noPrice)
positions['prob'] = positions.myBet.where(positions.purchase == 'y', 1 - positions.myBet)
positions

/tmp/ipykernel_17419/3622336434.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positions['price'] = positions.yesPrice.where(positions.purchase == 'y', positions.noPrice)
/tmp/ipykernel_17419/3622336434.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positions['prob'] = positions.myBet.where(positions.purchase == 'y', 1 - positions.myBet)


,symbol,date,yesPrice,noPrice,myBet,purchase,qty,price,prob
6,voch-001,2021-09-14,0.36,0.66,0.25,n,98.0,0.66,0.75
7,nyschl-001,2021-09-14,0.84,0.17,0.90,y,125.0,0.84,0.90


In [10]:
exp, situations = expected_value.compute_wager_expectations(positions.prob.values, positions.price.values, positions.qty.values)

In [11]:
exp

16.320000000000007

In [12]:
situations

,hits,hit_percent,hits_p,hits_exp
0,0,0.0,0.025,-169.68
1,1,0.5,0.300,-51.43
2,2,1.0,0.675,53.32
